In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from sklearn.utils import shuffle

The load_data() function takes the main 'path' as input, and using nested for-loops accesses the images present inside the subfolders of the folder which the path points to.  The accessed images are read using opencv module and stored inside img_data array. The label array stores the names of the subfolders as labels.

img_data is converted into a numpy array, and an extra _dimension_ is added(L 27) 
label_data uses the to_one_hot() function to create a suitable label array(L 29)

In [2]:
def load_data(path):
    
    sub_paths = os.listdir(path)

    img_data = []
    label_data = []
    
    c = 0
    
    for sub_path in sub_paths:
        
        img_paths = os.listdir(path + sub_path)
        
        for img_path in img_paths:
            
            img = cv2.imread(path + sub_path + '/' + img_path , 0)

            label = int(sub_path)
            
            img_data.append(img)
            label_data.append(label)
            
            c = c + 1
            
        print (sub_path)
        
    img_data = np.expand_dims(np.array(img_data),-1) #converting  to a numpy array
    #label_data = tf.one_hot(label_data, 10).eval(session = sess) # one_hot representation
    label_data = to_one_hot(label_data, 10)
    
    img_data, label_data = shuffle(img_data, label_data, random_state=0)
    return img_data, label_data


In [ ]:
for i in range(0,10):
    print('')

In [4]:
def to_one_hot(label_data, num_class):
    num_sample = np.shape(label_data)[0]
    temp = np.zeros([num_sample, num_class])
    temp[np.arange(num_sample),label_data] = 1
    return temp

In [5]:
def Network(Input): #input : [Batch_size, 32, 32, 1]
    with tf.name_scope("Network"):
        conv1 = tf.layers.conv2d(Input, filters = 64, kernel_size = 3, strides = 1, activation = tf.nn.relu, name = 'conv1')
        conv2 = tf.layers.conv2d(conv1, filters = 64, kernel_size = 3, strides = 1, activation = tf.nn.relu, name = 'conv2')
        pool1 = tf.layers.max_pooling2d(conv2, pool_size = 2, strides = 2, name = 'pool1')

        conv3 = tf.layers.conv2d(pool1, filters = 256, kernel_size = 3, strides = 1, activation = tf.nn.relu, name = 'conv3')
        conv4 = tf.layers.conv2d(conv3, filters = 256, kernel_size = 3, strides = 1, activation = tf.nn.relu, name = 'conv4')
        pool2 = tf.layers.max_pooling2d(conv4, pool_size = 2, strides = 2, name = 'pool2')

        conv5 = tf.layers.conv2d(pool2, filters = 512, kernel_size = 3, strides = 1, activation = tf.nn.relu, name = 'conv5')

        flat = tf.contrib.layers.flatten(conv5)
        fc1 = tf.layers.dense(flat, units = 1024, activation = tf.nn.relu, name = 'fc1')
        fc2 = tf.layers.dense(fc1, units = 10, activation = None, name = 'fc2')
    
    return fc2
    
    

In [6]:
def loss_function(logit, Label):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logit, labels = Label))
    return loss

In [7]:
def Accuracy_Evaluate(prediction, Label):
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return correct_pred, accuracy

In [8]:
def main(train_data, train_label, 
#          no_of_epochs = 150000,
         no_of_epochs = 1,
         batchsize = 32):
    
    
    Input = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28, 1])
    Label = tf.placeholder(dtype = tf.float32, shape = [None, 10])
    
    logit = Network(Input)
    
    prediction = tf.nn.softmax(logit)
    
    loss = loss_function(logit, Label)
    
    correct_pred, accuracy = Accuracy_Evaluate(prediction, Label)
    
    
    optimiz = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(loss)

    init = tf.global_variables_initializer()
    sess.run(init)
 
    
    tf.summary.scalar('Loss_Value',loss)
    tf.summary.scalar('Accuracy',accuracy)
    
    print('Stteing up summary op...')
    summary_op = tf.summary.merge_all()
    
    print('Setting Up Saver...')
    summary_writer = tf.summary.FileWriter('./log_dir/', sess.graph)
    
    itr = 0
    for epoch in range(no_of_epochs):
        
        index = np.random.permutation(np.shape(train_data)[0])
        train_data = train_data[index, :, :, :]
        train_label = train_label[index, :]
        
        for idx in range(train_data.shape[0]//batchsize): 
            
            batchx = train_data[idx*batchsize : (idx + 1)*batchsize , :, :, :]
            batchy = train_label[idx*batchsize : (idx + 1)*batchsize, :]
            
            feed_dict = {Input : batchx , Label : batchy}
            
            _, train_loss, train_accuracy, summary_str = sess.run([optimiz, loss, accuracy, summary_op] , feed_dict )
            summary_writer.add_summary(summary_str, itr)
            itr = itr + 1
            
            if idx%10 == 0:
                
                print ('epoch : '+str(epoch)+' step : '+str(idx) + ' train_loss : '+str(train_loss) +
                        ' train_accuracy : '+str(train_accuracy) 
            
                      )

In [9]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
global sess

config = tf.ConfigProto()
sess = tf.Session(config = config)
graph = tf.get_default_graph()

In [10]:
train_data, train_label = load_data('/home/abhirup/Desktop/Pycodes/Tf_Dl/trainingSet/')

0
6
8
1
3
2
9
4
5
7


In [15]:
train_data.shape

(42000, 28, 28, 1)

In [17]:
main(train_data, train_label)

Stteing up summary op...
Setting Up Saver...
epoch : 0 step : 0 train_loss : 2.66295 train_accuracy : 0.0625
epoch : 0 step : 10 train_loss : 0.304401 train_accuracy : 0.28125
epoch : 0 step : 20 train_loss : 0.266924 train_accuracy : 0.40625
epoch : 0 step : 30 train_loss : 0.232729 train_accuracy : 0.53125
epoch : 0 step : 40 train_loss : 0.207981 train_accuracy : 0.625
epoch : 0 step : 50 train_loss : 0.141598 train_accuracy : 0.9375
epoch : 0 step : 60 train_loss : 0.199252 train_accuracy : 0.625
epoch : 0 step : 70 train_loss : 0.180698 train_accuracy : 0.6875
epoch : 0 step : 80 train_loss : 0.127175 train_accuracy : 0.84375
epoch : 0 step : 90 train_loss : 0.119343 train_accuracy : 0.84375
epoch : 0 step : 100 train_loss : 0.124378 train_accuracy : 0.84375
epoch : 0 step : 110 train_loss : 0.141581 train_accuracy : 0.78125
epoch : 0 step : 120 train_loss : 0.111361 train_accuracy : 0.78125
epoch : 0 step : 130 train_loss : 0.100952 train_accuracy : 0.9375
epoch : 0 step : 140 tr

epoch : 0 step : 1200 train_loss : 0.0910231 train_accuracy : 0.84375
epoch : 0 step : 1210 train_loss : 0.0594725 train_accuracy : 0.90625
epoch : 0 step : 1220 train_loss : 0.0491435 train_accuracy : 0.96875
epoch : 0 step : 1230 train_loss : 0.0499844 train_accuracy : 0.875
epoch : 0 step : 1240 train_loss : 0.0237242 train_accuracy : 1.0
epoch : 0 step : 1250 train_loss : 0.0176282 train_accuracy : 1.0
epoch : 0 step : 1260 train_loss : 0.0310394 train_accuracy : 0.96875
epoch : 0 step : 1270 train_loss : 0.0564827 train_accuracy : 0.90625
epoch : 0 step : 1280 train_loss : 0.0510397 train_accuracy : 0.9375
epoch : 0 step : 1290 train_loss : 0.0432021 train_accuracy : 0.96875
epoch : 0 step : 1300 train_loss : 0.0202821 train_accuracy : 1.0
epoch : 0 step : 1310 train_loss : 0.0438795 train_accuracy : 1.0


In [ ]:
# Resource
# https://stackoverflow.com/questions/29831489/numpy-1-hot-array